In [1]:
import pandas as pd
import numpy as np

In [3]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

In [ ]:
from config import (
    N_CUSTOMERS,
    NUM_WEEKS,
    K,
    DECAY_RATE,
    RECOVERY_RATE
)

In [ ]:
df = pd.DataFrame({
    "Customer_id" : range(N_CUSTOMERS),
    "tolerance" : np.random.uniform(0.5,1.5,size = N_CUSTOMERS)
})
df['time_gap'] = np.random.exponential(scale=df['tolerance']*5)
df.head()

,Customer_id,tolerance,time_gap
0,0,0.759345,0.593082
1,1,1.422342,3.034544
2,2,1.223072,4.462016
3,3,1.455879,17.076588
4,4,1.136597,0.904445


In [ ]:
df['prev_time_gap'] = df['time_gap']

In [ ]:
K = 10
reviewed_ids = np.random.choice(df.index, size=K, replace = False)
df['reviewed'] = False
df.loc[reviewed_ids,'reviewed'] = True

In [ ]:
df.loc[df['reviewed']].head()

,Customer_id,tolerance,time_gap,prev_time_gap,reviewed
1,1,1.422342,3.034544,3.034544,True
12,12,1.446438,16.811858,16.811858,True
18,18,0.750959,0.242000,0.242000,True
20,20,1.448045,11.306405,11.306405,True
36,36,0.781056,5.441937,5.441937,True


In [ ]:
df.head()

,Customer_id,tolerance,time_gap,prev_time_gap,reviewed
0,0,0.759345,0.593082,0.593082,False
1,1,1.422342,3.034544,3.034544,True
2,2,1.223072,4.462016,4.462016,False
3,3,1.455879,17.076588,17.076588,False
4,4,1.136597,0.904445,0.904445,False


In [ ]:
df.loc[df['reviewed'],'tolerance'] = (df.loc[df['reviewed'],'tolerance'] * (1 + RECOVERY_RATE))

df.loc[~df['reviewed'],'tolerance'] = (df.loc[~df['reviewed'],'tolerance'] * (1 - DECAY_RATE))

df['time_gap'] = np.random.exponential(scale=df['tolerance'] * 5)

df.head()

,Customer_id,tolerance,time_gap,prev_time_gap,reviewed
0,0,0.607476,2.777381,0.593082,False
1,1,1.706811,7.136706,3.034544,True
2,2,0.978458,6.025615,4.462016,False
3,3,1.164703,0.376455,17.076588,False
4,4,0.909278,0.978262,0.904445,False


In [ ]:
df.loc[df['reviewed']].head()

,Customer_id,tolerance,time_gap,prev_time_gap,reviewed
1,1,1.706811,7.136706,3.034544,True
12,12,1.735726,3.776562,16.811858,True
18,18,0.901151,26.951381,0.242000,True
20,20,1.737654,21.941682,11.306405,True
36,36,0.937268,6.773959,5.441937,True


In [ ]:
df['gap_change'] = df['prev_time_gap'] - df['time_gap']
df.head()

,Customer_id,tolerance,time_gap,prev_time_gap,reviewed,gap_change
0,0,0.607476,2.777381,0.593082,False,-2.184299
1,1,1.706811,7.136706,3.034544,True,-4.102163
2,2,0.978458,6.025615,4.462016,False,-1.563599
3,3,1.164703,0.376455,17.076588,False,16.700133
4,4,0.909278,0.978262,0.904445,False,-0.073818


In [ ]:
df['worsening_streak'] = 0

In [ ]:
df.loc[df['gap_change'] > 0,'worsening_streak'] += 1
df.loc[df['gap_change']<=0,'worsening_streak'] = 0

In [ ]:
df.head()

,Customer_id,tolerance,time_gap,prev_time_gap,reviewed,gap_change,worsening_streak
0,0,0.607476,2.777381,0.593082,False,-2.184299,0
1,1,1.706811,7.136706,3.034544,True,-4.102163,0
2,2,0.978458,6.025615,4.462016,False,-1.563599,0
3,3,1.164703,0.376455,17.076588,False,16.700133,1
4,4,0.909278,0.978262,0.904445,False,-0.073818,0


In [ ]:
df['urgency_score'] = df['gap_change'] * df['worsening_streak']

In [ ]:
df.head()

,Customer_id,tolerance,time_gap,prev_time_gap,reviewed,gap_change,worsening_streak,urgency_score
0,0,0.607476,2.777381,0.593082,False,-2.184299,0,-0.000000
1,1,1.706811,7.136706,3.034544,True,-4.102163,0,-0.000000
2,2,0.978458,6.025615,4.462016,False,-1.563599,0,-0.000000
3,3,1.164703,0.376455,17.076588,False,16.700133,1,16.700133
4,4,0.909278,0.978262,0.904445,False,-0.073818,0,-0.000000


In [ ]:
df = df.sort_values('urgency_score',ascending=False)
to_review = df.head(K)
to_review

,Customer_id,tolerance,time_gap,prev_time_gap,reviewed,gap_change,worsening_streak,urgency_score
78,78,1.187678,10.978173,28.863924,False,17.885751,1,17.885751
3,3,1.164703,0.376455,17.076588,False,16.700133,1,16.700133
9,9,0.428317,0.719864,16.829002,False,16.109138,1,16.109138
81,81,1.084058,3.163698,17.634296,False,14.470598,1,14.470598
95,95,0.604438,0.166853,13.483299,False,13.316445,1,13.316445
12,12,1.735726,3.776562,16.811858,True,13.035296,1,13.035296
42,42,0.562966,1.424480,12.954158,False,11.529678,1,11.529678
27,27,0.830918,0.714459,8.917544,False,8.203085,1,8.203085
61,61,0.487298,0.347342,7.721435,False,7.374093,1,7.374093
68,68,0.604067,0.377970,7.130470,False,6.752500,1,6.752500
